<a href="https://colab.research.google.com/github/mdeniz1/datascienceprojects/blob/main/Multi_Aspect_Based_Classification_using_LSTM_based_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Multi-aspect based classification is performed in this notebook. Essentially, it works on a LSTM-based neural network model for multi-label aspect classification.

In the training set, there can be multiple aspects for a sentence. Therefore, a model needs to predict multiple labels simultaneously or multiple models need to be created for multiple targets. I followed the first approach.

To summarize what I have done in the notebook:

Data cleaning and encoding:
I noticed that the aspects in the training data were incorrectly encoded. For example, many aspects actually represented similar contexts. For instance, aspects like 'personnel,' 'service,' and 'attention' essentially provided information about the service received at a location. Considering the large dataset and the risk of losing nuances between these aspects, I decided to group them based on their meanings. I merged several aspects together. Additionally, I removed aspects with less than 25 occurrences in the dataset as trying to predict over 200 targets from a small dataset of around 900 instances would not yield good results.
Training a multi-label bi-LSTM model and showcasing the results.
I used the test set for testing the model, following the data cleaning and transformation process I performed in the first step.

In [ ]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense
import requests
from sklearn.model_selection import train_test_split
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from sklearn.metrics import precision_score, recall_score





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2.12.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:


url = 'https://github.com/yangheng95/ABSADatasets/raw/v2.0/datasets/apc_datasets/120.SemEval2016Task5/128.turkish/restaurants_train_turkish.xml.dat'
response = requests.get(url)
lines = response.text.splitlines()

texts = []
aspects = []
sentiments = []

for i in range(0, len(lines), 3):
    text = lines[i].strip()
    aspect = lines[i + 1].strip()
    sentiment = lines[i + 2].strip()

    texts.append(text)
    aspects.append(aspect)
    sentiments.append(sentiment)

data = {'text': texts, 'aspect': aspects, 'sentiment': sentiments}
df = pd.DataFrame(data)

df['text'] = df.apply(lambda row: row['text'].replace('$T$', str(row['aspect'])), axis=1)




In [ ]:
df['text'] = df['text'].str.lower()
df['aspect'] = df['aspect'].str.lower()
df['text'] = df['text'].str.replace(r"'", '')
df['aspect'] = df['aspect'].str.replace(r"'", '')
df['text'] = df['text'].str.replace(r"-", '')
df['aspect'] = df['aspect'].str.replace(r"-", '')
df['text'] = df['text'].str.replace(r"(", '')
df['aspect'] = df['aspect'].str.replace(r"(", '')
df['text'] = df['text'].str.replace(r")", '')
df['aspect'] = df['aspect'].str.replace(r")", '')


<ipython-input-3-ac392c51a14c>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['text'] = df['text'].str.replace(r"(", '')
<ipython-input-3-ac392c51a14c>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['aspect'] = df['aspect'].str.replace(r"(", '')
<ipython-input-3-ac392c51a14c>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['text'] = df['text'].str.replace(r")", '')
<ipython-input-3-ac392c51a14c>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, si

In [ ]:
aspect_num=df['aspect'].nunique()
aspect_num

838

In [ ]:
df = df.drop('sentiment', axis=1)
df.drop(691, axis=0, inplace=True) # similar aspect for the same sentence
df.drop(732, axis=0, inplace=True) #repeating aspect for the same sentence





In [ ]:
df['aspect'] = df['aspect'].astype(str)
df['text'] = df['text'].astype(str)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1383 entries, 0 to 1384
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1383 non-null   object
 1   aspect  1383 non-null   object
dtypes: object(2)
memory usage: 32.4+ KB


I applied manual lemmatization. After observing that Turkish lemmatizers were providing incorrect results, I decided to resort to this approach.

In [ ]:
import re

lemma = ['ortam', 'manzara', 'yemek', 'kalite', 'yemegi', 'fiyat', 'garson', 'çay', 'salata', 'guvec', 'sube', 'köşe', 'pilav','mekan','ürün','sütlac',
        'sos','lokum', 'servis', 'sütlaç','porsiyon','lezzet', 'beşiktaş', 'lezzet','sipariş','kenar', 'orta', 'bütçe', 'yer', 'meze',
         'kağıt', 'köprü', 'ışık','hisar','müzik', 'döner', 'et', 'dilim', 'wrap','tavuk','tad', 'tuvalet', 'eleman', 'servis', 'spesyal',
         'tatlı','kuaför','restoran', 'hanım', 'getir', 'bölüm', 'bey','işletmeci',
         'baklava', 'bura', 'geçir', 'çalışan','ilgi','hamburger','porsiyon','seçim','çeşit','fasulye','fasülye','kahvaltı',
         'koku','döner','yoğurt','bütçe','kızartma','ara','hizmet', 'merkez','bira','magnolia','ızdara','dekorasyon','seçenek', 'gel',
         'kesim','lavas','kebap','kebab', 'arkadaş','yer','aroma', 'sufle','ikram','parça','halk', 'manti','mantı','ayran','konum','hesap','paket', 'malzeme', 'boğaz',
         'başlangıç', 'kart','helva','sis','lahmacun','balıkçı','dükkan','tereyağ','künefe','salep','bilgilendirme','nargile', 'menü','tavır','musteri','müessese', 'masa','konum',
         'vale','sunum','çanak','tavuk göğsü','içecek','işletme','aşk-ı memnu', 'fıccın', 'yumurta','bal','sokak','ev','aperitif','aperatif','nargile','biber','personel',
         'terbiyesiz','renk','ayran', 'manti','mantı','bahçe','aşkı memnu','pizza','menemen','çorba','corba','sube','şube', 'bahce','alan','calisan','limon',
         'müşteri','dem','limonata','pasta','patates','su', 'puding','kalabalık','şey','kafe','ulaşım','bekleme','gecikme','pudding']
for replacement in lemma:
    pattern = fr'\b{replacement}\w*\b'
    df['aspect'] = df['aspect'].apply(lambda x: re.sub(pattern, replacement, x))


below is clustering of the aspects.

In [ ]:



word_pairs = {'işletmeci': 'sahibi', 'hesap': 'fiyat', 'dem':'çay','ekmeği':'ekmek','yediğimi':'yemek','bütçe':'fiyat','doyduk':'doymak','yemeğini':'yemek','merkez':'mekan',
              'salıncağı':'salıncak','kisiyorum':'kısmak','seçeneğiniz':'seçenek','fiyat skalasını':'fiyat', 'kebab':'kebap', 'yediklerimden':'yemek', 'e5 üstünde':'konum',
              'beklesekte':'gecikme', 'anzarası':'manzara', 'kalkmak':'kalabalık','gidilebilir':'mekan','sabah kahvaltısı':'kahvaltı', 'geniş bahçesi ve terası':'bahçe','nusretinki':'yemek',
            'kedi böreği':'börek','host ediyorlar':'servis','otagtepenin manzara':'manzara', 'kestanesi suflesi':'sufle', 'fresh meyve suları':'meyve suyu','boğazın göbeğinde':'konum',
              'manti':'mantı','serpme kahvaltı':'kahvaltı','çay veya kahve':'içecek', 'kahve':'içecek','çay':'içecek','canlı müzik':'müzik', 'lynchburgleri':'hamburger',
              '80/90 tl hesap':'hesap', 'İthal peynir tabağındaki peynirler':'peynir)', 'pahalanmış':'fiyat', 'üst kat terası':'teras', 'Patlıcanlı zeytinli penne':'penne','Sipariş almaya geç gel':'gecikme',
              'kurufasulye ve sütlaç':'yemek', 'çıkış yolu':'konum', '342 tl':'hesap','pide türleri':'çeşit','demlenen':'demlemek','ücrete':'fiyat','diyorlar':'demek',
              'musteri':'müşteri','müessese':'işletme', 'lavas':'lavaş','yediriyor':'yedirmek', 'hersey':'her şey','her sey':'her şey', 'garson ilgi':'servis', 'garson':'servis',
              'servis hızı':'servis', 'manzara parası':'fiyat', '13-14tl':'fiyat', 'ogle':'öğle', 'yer bulmak':'kalabalık','kirmizi limonata':'limonata', 'self servis':'servis', 'köfte':'köfte',
              'pilav da kurufasulye de':'yemek', 'pazar kahvaltısı':'kahvaltı', 'sohbet orta':'ortam', 'yemeği':'yemek', 'kafe yemek':'yemek', 'iç dizaynını':'tasarım', 'kış aylarında':'kış',
              'karamel ara çay':'çay', 'pizza fagotto':'pizza', 'sorbe dondurması':'dondurma', 'şatorbiryan isimli et':'et', 'mantar çorbası':'çorba', 'yiyecekleri':'yemek',
              'yer bulamamak':'kalabalık', 'kahvalti':'kahvaltı', 'lezzet kalitesi':'lezzet', 'mekan tasarımı':'tasarım', 'ikisi de':'yemek','Passion fruitlu dondurması':'dondurma',
              'aşşk-ı memnu':'aşkı-memnu', 'içindeki yağın':'yağ', 'gireceğimizi':'konum', 'cafe orta':'ortam', 'tavla, okey gibi oyunları':'oyun', 'yiyebilirsiniz':'yemek',
              'maçlarını':'maç', 'kuru fasulye lezzet':'lezzet', 'mekan konum':'konum','mekan kimliği enerjisi':'ortam', 'otoparkta masa':'ortam', 'mekan üst katlarinda':'mekan',
              'içi':'tasarım','servis eden':'servis', 'havuç dilimi':'baklava', 'el yapımı ayran':'ayran', 'küçük pizzaları':'pizza', 'domates sos':'sos',
              'bira menüsü':'menü', 'ice tea ve limonataları':'içecek', 'et meze':'meze', 'çay çeşidi':'çeşit','damla sakızlı mastika keki':'kek', 'dekorasyon ve orta':'ortam',
              'dekorasyon':'tasarım', 'çalışan':'garson','stres kontrolu':'kalabalık', 'istanbul şubesi':'şube', 'arka bahçesi':'bahçe','iyilerinden':'mekan', 'masa yer':'ortam',
              'yer bulmak':'kalabalık','tereyağlı pilav':'pilav', 'hizmet':'servis','bura':'mekan', 'personel':'servis','yer':'mekan','orta':'ortam','ev yemek':'yemek','küçük pizzaları':'pizza',
              '13.5 lira':'fiyat','4.5 lira':'fiyat','fıstıklı tat':'yemek',  'künefe':'dessert', 'baklava':'dessert','içki servis':'servis','patatesli mantı':'mantı', 'ev yemeği':'yemek', 'iç mekan konseptiyle':'tasarım',
               'corba':'çorba', 'kofte':'köfte','iç alanı':'ortam', 'ny steak':'et', 'ayran':'içecek','orta':'ortam','alan':'ortam','ikram':'servis', 'ayran':'içecek','alkol':'içecek', 'calisan':'servis','hesabı':'fiyat', 'burger':'hamburger',
              'limon':'limonata', 'yemege':'yemek', 'mayonez':'sos','musteri':'müşteri','cay':'çay','dondurma':'dessert','jager mojito':'içecek','yoğurdu':'yoğurt', 'tavirlarina':'hizmet', 'muamele':'hizmet', 'aşşkı memnu':'aşkı memnu', 'sote':'et',
              'limonata':'içecek','kavurmalı':'et', 'sarap':'şarap', 'şarap':'içecek','görselliğe':'tasarım','lakerdası':'balık','ıstakoz':'balık','su':'içecek','sahlep':'içecek','ambians':'ortam','havuç dilim':'dessert',
              'yenilen her şey':'yemek','dizaynı':'tasarım','kişi başı':'fiyat','somon':'balık','kapali kasarlisi':'pide', 'menu fiyat': 'fiyati','ev cafe':'cafe','ilgi':'servis', 'her şey':'yemek',
              'ulasım':'ulaşım','ulaşım':'konum','bekleme':'gecikme','d bal izgara':'balık','mantı ficcini zeytinyağlı mezgiti':'yemek','koltukları':'ortam','irish malt':'içecek','fiyatn gel':'gecikme',
              'gecikme':'servis','ev hanım':'müşteri','bira':'içecek','hamhamburger':'hamburger','lokasyon':'konum','pudding':'dessert', 'alkol':'içecek','aşkı memnu':'dessert', 'tatlı':'dessert', 'tl':'fiyat',
              'atıştırmalık jumbo combo':'yemek','kebap':'et','kisi basi':'fiyat'}

for word, replacement in word_pairs.items():
    df['aspect'] = df['aspect'].str.replace(word, replacement)



<ipython-input-8-97f1f5d0b8e2>:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['aspect'] = df['aspect'].str.replace(word, replacement)


In [ ]:
mylist=['pizza','çorba','tavuk','salata','patates','içecek','mekan','hamburger','biber','peynir','balık','müşteri','servis','penne', 'pide','ortam',
        'fastfood','bahce','alkol','pilav','yemek','şube','sube','kahvalti','zeytinyag','dessert','et','köfte','manzara','menü','tost','çorba','kofte','kebap','fiyat']
for aspect in mylist:
    df.loc[df['aspect'].str.contains(aspect, case=False, na=False), 'aspect'] = aspect

In [ ]:
df['aspect'] = df['aspect'].str.replace('dessert', 'tatlı')


df.loc[df['aspect'].str.contains('kuzu', case=False, na=False), 'aspect'] = 'et'
df.loc[df['aspect'].str.contains('teras', case=False, na=False), 'aspect'] = 'bahçe'
df.loc[df['aspect'].str.contains('yoğurt', case=False, na=False), 'aspect'] = 'yoğurt'
df.loc[df['aspect'].str.contains('yagindan', case=False, na=False), 'aspect'] = 'yemek'
df.loc[df['aspect'].str.contains('pasta', case=False, na=False), 'aspect'] = 'tatlı'
df.loc[df['aspect'].str.contains('havuç dilim', case=False, na=False), 'aspect'] = 'tatlı'
df.loc[df['aspect'].str.contains('yagin', case=False, na=False), 'aspect'] = 'yemek'
df.loc[df['aspect'].str.contains('yemegi', case=False, na=False), 'aspect'] = 'yemek'
df.loc[df['aspect'].str.contains('hahvaltı', case=False, na=False), 'aspect'] = 'kahvaltı'
df.loc[df['aspect'].str.contains('puding', case=False, na=False), 'aspect'] = 'tatlı'




In [ ]:
aspect_num=df['aspect'].nunique()
aspect_num

247

In [ ]:
df['aspect'] = df['aspect'].str.replace('hamhamburger', 'hamburger')
df['aspect'] = df['aspect'].str.replace('ortammm', 'ortam')


In [ ]:
df['aspect'].value_counts()

servis                          160
mekan                           140
yemek                           113
fiyat                           104
içecek                          103
et                               84
manzara                          69
ortam                            36
tatlı                            29
kahvalti                         20
konum                            20
meze                             19
pizza                            17
menü                             16
bahçe                            15
pide                             14
hamburger                        13
tasarım                          13
köfte                            13
tavuk                            12
balık                            11
porsiyon                         11
kalite                           10
müzik                            10
salata                            8
sos                               7
menemen                           7
patates                     

Despite intensive clustering efforts, there are still many aspects with fewer than 25 occurrences. Due to the small size of the dataset, I decided to remove these aspects.





In [ ]:

aspect_counts = df['aspect'].value_counts()

aspects_to_drop = aspect_counts[aspect_counts < 25].index
aspects_to_drop

df2 = df[~df['aspect'].isin(aspects_to_drop)]


In [ ]:
df2['aspect'].nunique()

9

In [ ]:
df2=df2.reset_index()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   812 non-null    int64 
 1   text    812 non-null    object
 2   aspect  812 non-null    object
dtypes: int64(1), object(2)
memory usage: 19.2+ KB


In [ ]:
df2.drop('index', axis=1, inplace=True)

In [ ]:
df2.head()

,text,aspect
0,manzara sahane evet ama servis rezalet.,manzara
1,manzara sahane evet ama servis rezalet.,servis
2,"soguk su isteyince, soguk yok, butun sulari di...",servis
3,"yemekler iyi hos, lezzetler iyi ama heyecan ve...",yemek
4,"yemekler iyi hos, lezzetler iyi ama heyecan ve...",et


In [ ]:
encoded_df = pd.get_dummies(df2, columns=['aspect'])


In [ ]:
encoded_df.head()

,text,aspect_et,aspect_fiyat,aspect_içecek,aspect_manzara,aspect_mekan,aspect_ortam,aspect_servis,aspect_tatlı,aspect_yemek
0,manzara sahane evet ama servis rezalet.,0,0,0,1,0,0,0,0,0
1,manzara sahane evet ama servis rezalet.,0,0,0,0,0,0,1,0,0
2,"soguk su isteyince, soguk yok, butun sulari di...",0,0,0,0,0,0,1,0,0
3,"yemekler iyi hos, lezzetler iyi ama heyecan ve...",0,0,0,0,0,0,0,0,1
4,"yemekler iyi hos, lezzetler iyi ama heyecan ve...",1,0,0,0,0,0,0,0,0


In [ ]:
encoded_df = pd.get_dummies(df2, columns=['aspect'])
df_model = encoded_df.groupby('text').sum().reset_index()


In [ ]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 635 entries, 0 to 634
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            635 non-null    object
 1   aspect_et       635 non-null    uint8 
 2   aspect_fiyat    635 non-null    uint8 
 3   aspect_içecek   635 non-null    uint8 
 4   aspect_manzara  635 non-null    uint8 
 5   aspect_mekan    635 non-null    uint8 
 6   aspect_ortam    635 non-null    uint8 
 7   aspect_servis   635 non-null    uint8 
 8   aspect_tatlı    635 non-null    uint8 
 9   aspect_yemek    635 non-null    uint8 
dtypes: object(1), uint8(9)
memory usage: 10.7+ KB


building the model

In [ ]:
num_targets=9
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(df_model['text'])
sequences = tokenizer.texts_to_sequences(df_model['text'])

max_sequence_length = max(len(sequence) for sequence in sequences)
vocab_size2=tokenizer.num_words
embedding_dim=100
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
print(vocab_size2)
targets = df_model.iloc[:, 1:].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, targets, test_size=0.05, random_state=42)


In [ ]:
y_train_reshaped = np.repeat(y_train[:, np.newaxis, :], max_sequence_length, axis=1)
y_test_reshaped = np.repeat(y_test[:, np.newaxis, :], max_sequence_length, axis=1)

input_layer = Input(shape=(max_sequence_length,))

embedding_layer = Embedding(input_dim=vocab_size2, output_dim=embedding_dim)(input_layer)

lstm_layer = LSTM(units=1024, return_sequences=True)(embedding_layer)
dropout_layer = Dropout(0.65)(lstm_layer)


dense_layer_2 = Dense(units=num_targets, kernel_regularizer=l2(0.009), activation='sigmoid')(dropout_layer)

model = tf.keras.Model(inputs=input_layer, outputs=dense_layer_2)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['Precision','Recall',])


model.fit(X_train, y_train_reshaped, validation_data=(X_test, y_test_reshaped), epochs=65, batch_size=32)


Epoch 1/65
19/19 [==============================] - 6s 173ms/step - loss: 0.6231 - precision: 0.1637 - recall: 0.0443 - val_loss: 0.5368 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/65
19/19 [==============================] - 2s 101ms/step - loss: 0.5102 - precision: 0.2323 - recall: 0.0022 - val_loss: 0.4945 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/65
19/19 [==============================] - 1s 67ms/step - loss: 0.4715 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.4846 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/65
19/19 [==============================] - 1s 35ms/step - loss: 0.4491 - precision: 1.0000 - recall: 3.1216e-05 - val_loss: 0.4506 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/65
19/19 [==============================] - 1s 50ms/step - loss: 0.4350 - precision: 0.4000 - recall: 2.4973e-04 - val_loss: 0.4496 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 6/65
19/19 [===================

I developed a model by adjusting hyperparameters and changing the number of layers to prevent overfitting with the available data. The training loss was 0.28, and the validation loss was 0.45, yielding similar results. However, when examining the metrics, both precision and recall appear to be low. I would like to highlight two points:

It's important to remember that accuracy measures whether an instance is predicted entirely correct or not. In this case, the prediction result is an array of probabilities, and the accuracy metric checks for an exact match between this array and the target array. It doesn't consider the matching ratio of individual labels within the array. Therefore, I believe that considering label-level matching would yield better results. Unfortunately, as of now, Keras does not provide a specific metric like "precision per item" for multi-label problems, so I couldn't use it.
The dataset is very small, which limits its ability to generalize. We will see this issue more clearly when evaluating the model on the test set.
In summary, the model achieved relatively low precision and recall due to the limitations of the dataset size and the chosen evaluation metrics.





In [ ]:
model.save("model.keras")

In [ ]:
model2 = load_model("model.keras")


In this section, I am applying the same data cleaning process that I performed on the training set to the test set.

In [ ]:
import pandas as pd
import requests
import re

url = "https://github.com/yangheng95/ABSADatasets/raw/v2.0/datasets/apc_datasets/120.SemEval2016Task5/128.turkish/restaurants_test_turkish.xml.dat.inference"

# Read the file from the URL
response = requests.get(url)
data = response.content.decode("utf-8")

# Create a DataFrame with the desired columns
df = pd.DataFrame(columns=["text", "aspect", "sentiment"])

# Process each line of the file
for line in data.split("\n"):
    if line:
        matches = re.match(r"(.*?)\[B-ASP\](.*?)\[E-ASP\](.*?)\$LABEL\$(.*)", line)
        if matches:
            text = matches.group(1) + matches.group(2) + matches.group(3)
            aspect = matches.group(2)
            sentiment = matches.group(4)
            df = df.append({"text": text.strip(), "aspect": aspect.strip(), "sentiment": sentiment.strip()}, ignore_index=True)

# Display the resulting DataFrame


In [ ]:
df.head()

,text,aspect,sentiment
0,Serviste cok guzel.,Serviste,Positive
1,Ortam cok guzel.,Ortam,Positive
2,En beğendigimiz sushico şubesi.,sushico şubesi,Positive
3,Bahçesi çok güzel ve ferah.,Bahçesi,Positive
4,Sushi ustasi cok becerikli ve şakaci :,Sushi ustasi,Positive


In [ ]:
df['text'] = df['text'].str.lower()
df['aspect'] = df['aspect'].str.lower()
df['text'] = df['text'].str.replace(r"'", '')
df['aspect'] = df['aspect'].str.replace(r"'", '')
df['text'] = df['text'].str.replace(r"-", '')
df['aspect'] = df['aspect'].str.replace(r"-", '')
df['text'] = df['text'].str.replace(r"(", '')
df['aspect'] = df['aspect'].str.replace(r"(", '')
df['text'] = df['text'].str.replace(r")", '')
df['aspect'] = df['aspect'].str.replace(r")", '')


In [ ]:
aspect_num=df['aspect'].nunique()
aspect_num

117

In [ ]:
df = df.drop('sentiment', axis=1)


In [ ]:
df['aspect'] = df['aspect'].astype(str)
df['text'] = df['text'].astype(str)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    145 non-null    object
 1   aspect  145 non-null    object
dtypes: object(2)
memory usage: 2.4+ KB


In [ ]:
df['aspect'].value_counts()

mekan                           7
servis                          5
yemekler                        3
etler                           3
fiyatlar                        3
garsonlar                       3
lezzetler                       2
saç arasi                       2
döner                           2
çalışanları                     2
kahvalti                        2
biçak arasi                     2
ambiyans                        2
çalışanlar                      2
yemekleri                       2
garsonlarin                     2
kebapları                       1
kuzu kol                        1
zeytinyağlılar                  1
salonlar                        1
imambayıldı                     1
konumu                          1
lahmacun                        1
manzarası                       1
lezzeti                         1
fiyatlara                       1
garsonlari                      1
beyoglunda ki subede            1
9,5 tl                          1
pilavda       

In [ ]:
import re

lemma = ['ortam', 'manzara', 'yemek', 'kalite', 'yemegi', 'fiyat', 'garson', 'çay', 'salata', 'guvec', 'sube', 'köşe', 'pilav','mekan','ürün','sütlac',
        'sos','lokum', 'servis', 'sütlaç','porsiyon','lezzet', 'beşiktaş', 'lezzet','sipariş','kenar', 'orta', 'bütçe', 'yer', 'meze',
         'kağıt', 'köprü', 'ışık','hisar','müzik', 'döner', 'et', 'dilim', 'wrap','tavuk','tad', 'tuvalet', 'eleman', 'servis', 'spesyal',
         'tatlı','kuaför','restoran', 'hanım', 'getir', 'bölüm', 'bey','işletmeci',
         'baklava', 'bura', 'geçir', 'çalışan','ilgi','hamburger','porsiyon','seçim','çeşit','fasulye','fasülye','kahvaltı',
         'koku','döner','yoğurt','bütçe','kızartma','ara','hizmet', 'merkez','bira','magnolia','ızdara','dekorasyon','seçenek', 'gel',
         'kesim','lavas','kebap','kebab', 'arkadaş','yer','aroma', 'sufle','ikram','parça','halk', 'manti','mantı','ayran','konum','hesap','paket', 'malzeme', 'boğaz',
         'başlangıç', 'kart','helva','sis','lahmacun','balıkçı','dükkan','tereyağ','künefe','salep','bilgilendirme','nargile', 'menü','tavır','musteri','müessese', 'masa','konum',
         'vale','sunum','çanak','tavuk göğsü','içecek','işletme','aşk-ı memnu', 'fıccın', 'yumurta','bal','sokak','ev','aperitif','aperatif','nargile','biber','personel',
         'terbiyesiz','renk','ayran', 'manti','mantı','bahçe','aşkı memnu','pizza','menemen','çorba','corba','sube','şube', 'bahce','alan','calisan','limon',
         'müşteri','dem','limonata','pasta','patates','su', 'puding','kalabalık','şey','kafe','ulaşım','bekleme','gecikme','pudding']
for replacement in lemma:
    pattern = fr'\b{replacement}\w*\b'
    df['aspect'] = df['aspect'].apply(lambda x: re.sub(pattern, replacement, x))


In [ ]:



word_pairs = {'işletmeci': 'sahibi', 'hesap': 'fiyat', 'dem':'çay','ekmeği':'ekmek','yediğimi':'yemek','bütçe':'fiyat','doyduk':'doymak','yemeğini':'yemek','merkez':'mekan',
              'salıncağı':'salıncak','kisiyorum':'kısmak','seçeneğiniz':'seçenek','fiyat skalasını':'fiyat', 'kebab':'kebap', 'yediklerimden':'yemek', 'e5 üstünde':'konum',
              'beklesekte':'gecikme', 'anzarası':'manzara', 'kalkmak':'kalabalık','gidilebilir':'mekan','sabah kahvaltısı':'kahvaltı', 'geniş bahçesi ve terası':'bahçe','nusretinki':'yemek',
            'kedi böreği':'börek','host ediyorlar':'servis','otagtepenin manzara':'manzara', 'kestanesi suflesi':'sufle', 'fresh meyve suları':'meyve suyu','boğazın göbeğinde':'konum',
              'manti':'mantı','serpme kahvaltı':'kahvaltı','çay veya kahve':'içecek', 'kahve':'içecek','çay':'içecek','canlı müzik':'müzik', 'lynchburgleri':'hamburger',
              '80/90 tl hesap':'hesap', 'İthal peynir tabağındaki peynirler':'peynir)', 'pahalanmış':'fiyat', 'üst kat terası':'teras', 'Patlıcanlı zeytinli penne':'penne','Sipariş almaya geç gel':'gecikme',
              'kurufasulye ve sütlaç':'yemek', 'çıkış yolu':'konum', '342 tl':'hesap','pide türleri':'çeşit','demlenen':'demlemek','ücrete':'fiyat','diyorlar':'demek',
              'musteri':'müşteri','müessese':'işletme', 'lavas':'lavaş','yediriyor':'yedirmek', 'hersey':'her şey','her sey':'her şey', 'garson ilgi':'servis', 'garson':'servis',
              'servis hızı':'servis', 'manzara parası':'fiyat', '13-14tl':'fiyat', 'ogle':'öğle', 'yer bulmak':'kalabalık','kirmizi limonata':'limonata', 'self servis':'servis', 'köfte':'köfte',
              'pilav da kurufasulye de':'yemek', 'pazar kahvaltısı':'kahvaltı', 'sohbet orta':'ortam', 'yemeği':'yemek', 'kafe yemek':'yemek', 'iç dizaynını':'tasarım', 'kış aylarında':'kış',
              'karamel ara çay':'çay', 'pizza fagotto':'pizza', 'sorbe dondurması':'dondurma', 'şatorbiryan isimli et':'et', 'mantar çorbası':'çorba', 'yiyecekleri':'yemek',
              'yer bulamamak':'kalabalık', 'kahvalti':'kahvaltı', 'lezzet kalitesi':'lezzet', 'mekan tasarımı':'tasarım', 'ikisi de':'yemek','Passion fruitlu dondurması':'dondurma',
              'aşşk-ı memnu':'aşkı-memnu', 'içindeki yağın':'yağ', 'gireceğimizi':'konum', 'cafe orta':'ortam', 'tavla, okey gibi oyunları':'oyun', 'yiyebilirsiniz':'yemek',
              'maçlarını':'maç', 'kuru fasulye lezzet':'lezzet', 'mekan konum':'konum','mekan kimliği enerjisi':'ortam', 'otoparkta masa':'ortam', 'mekan üst katlarinda':'mekan',
              'içi':'tasarım','servis eden':'servis', 'havuç dilimi':'baklava', 'el yapımı ayran':'ayran', 'küçük pizzaları':'pizza', 'domates sos':'sos',
              'bira menüsü':'menü', 'ice tea ve limonataları':'içecek', 'et meze':'meze', 'çay çeşidi':'çeşit','damla sakızlı mastika keki':'kek', 'dekorasyon ve orta':'ortam',
              'dekorasyon':'tasarım', 'çalışan':'garson','stres kontrolu':'kalabalık', 'istanbul şubesi':'şube', 'arka bahçesi':'bahçe','iyilerinden':'mekan', 'masa yer':'ortam',
              'yer bulmak':'kalabalık','tereyağlı pilav':'pilav', 'hizmet':'servis','bura':'mekan', 'personel':'servis','yer':'mekan','orta':'ortam','ev yemek':'yemek','küçük pizzaları':'pizza',
              '13.5 lira':'fiyat','4.5 lira':'fiyat','fıstıklı tat':'yemek',  'künefe':'dessert', 'baklava':'dessert','içki servis':'servis','patatesli mantı':'mantı', 'ev yemeği':'yemek', 'iç mekan konseptiyle':'tasarım',
               'corba':'çorba', 'kofte':'köfte','iç alanı':'ortam', 'ny steak':'et', 'ayran':'içecek','orta':'ortam','alan':'ortam','ikram':'servis', 'ayran':'içecek','alkol':'içecek', 'calisan':'servis','hesabı':'fiyat', 'burger':'hamburger',
              'limon':'limonata', 'yemege':'yemek', 'mayonez':'sos','musteri':'müşteri','cay':'çay','dondurma':'dessert','jager mojito':'içecek','yoğurdu':'yoğurt', 'tavirlarina':'hizmet', 'muamele':'hizmet', 'aşşkı memnu':'aşkı memnu', 'sote':'et',
              'limonata':'içecek','kavurmalı':'et', 'sarap':'şarap', 'şarap':'içecek','görselliğe':'tasarım','lakerdası':'balık','ıstakoz':'balık','su':'içecek','sahlep':'içecek','ambians':'ortam','havuç dilim':'dessert',
              'yenilen her şey':'yemek','dizaynı':'tasarım','kişi başı':'fiyat','somon':'balık','kapali kasarlisi':'pide', 'menu fiyat': 'fiyati','ev cafe':'cafe','ilgi':'servis', 'her şey':'yemek',
              'ulasım':'ulaşım','ulaşım':'konum','bekleme':'gecikme','d bal izgara':'balık','mantı ficcini zeytinyağlı mezgiti':'yemek','koltukları':'ortam','irish malt':'içecek','fiyatn gel':'gecikme',
              'gecikme':'servis','ev hanım':'müşteri','bira':'içecek','hamhamburger':'hamburger','lokasyon':'konum','pudding':'dessert', 'alkol':'içecek','aşkı memnu':'dessert', 'tatlı':'dessert', 'tl':'fiyat',
              'atıştırmalık jumbo combo':'yemek','kebap':'et','kisi basi':'fiyat'}

for word, replacement in word_pairs.items():
    df['aspect'] = df['aspect'].str.replace(word, replacement)



<ipython-input-48-97f1f5d0b8e2>:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['aspect'] = df['aspect'].str.replace(word, replacement)


In [ ]:
mylist=['pizza','çorba','tavuk','salata','patates','içecek','mekan','hamburger','biber','peynir','balık','müşteri','servis','penne', 'pide','ortam',
        'fastfood','bahce','alkol','pilav','yemek','şube','sube','kahvalti','zeytinyag','dessert','et','köfte','manzara','menü','tost','çorba','kofte','kebap','fiyat']
for aspect in mylist:
    df.loc[df['aspect'].str.contains(aspect, case=False, na=False), 'aspect'] = aspect

In [ ]:
df['aspect'] = df['aspect'].str.replace('dessert', 'tatlı')


df.loc[df['aspect'].str.contains('kuzu', case=False, na=False), 'aspect'] = 'et'
df.loc[df['aspect'].str.contains('teras', case=False, na=False), 'aspect'] = 'bahçe'
df.loc[df['aspect'].str.contains('yoğurt', case=False, na=False), 'aspect'] = 'yoğurt'
df.loc[df['aspect'].str.contains('yagindan', case=False, na=False), 'aspect'] = 'yemek'
df.loc[df['aspect'].str.contains('pasta', case=False, na=False), 'aspect'] = 'tatlı'
df.loc[df['aspect'].str.contains('havuç dilim', case=False, na=False), 'aspect'] = 'tatlı'
df.loc[df['aspect'].str.contains('yagin', case=False, na=False), 'aspect'] = 'yemek'
df.loc[df['aspect'].str.contains('yemegi', case=False, na=False), 'aspect'] = 'yemek'
df.loc[df['aspect'].str.contains('hahvaltı', case=False, na=False), 'aspect'] = 'kahvaltı'
df.loc[df['aspect'].str.contains('puding', case=False, na=False), 'aspect'] = 'tatlı'




In [ ]:
df['aspect'].value_counts()

servis                          18
et                              17
fiyat                           14
içecek                          12
mekan                           12
yemek                           10
tatlı                            6
döner                            3
manzara                          3
kahvalti                         3
tavuk                            2
şube                             2
ambiyans                         2
bahçe                            2
pilav                            2
saç ara                          2
biçak ara                        2
iskenderi                        1
sipariş                          1
lahmacun                         1
salonlar                         1
zeytinyağlılar                   1
imambayıldı                      1
lokanta                          1
ambiyans ve tasarım              1
konum                            1
sos                              1
porsiyon miktarı                 1
yogurt              

To create a test dataset with the same target format as the model, it is necessary to convert the test data into a 9-element array format.

In [ ]:

aspect_counts = df['aspect'].value_counts()

aspects_to_drop = aspect_counts[aspect_counts < 3].index
aspects_to_drop

df2 = df[~df['aspect'].isin(aspects_to_drop)]
df2=df2.reset_index()
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   98 non-null     int64 
 1   text    98 non-null     object
 2   aspect  98 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [ ]:
df2.drop('index', axis=1, inplace=True)

In [ ]:
encoded_df = pd.get_dummies(df2, columns=['aspect'])
df_test = encoded_df.groupby('text').sum().reset_index()


In [ ]:
df_test.head()

,text,aspect_döner,aspect_et,aspect_fiyat,aspect_içecek,aspect_kahvalti,aspect_manzara,aspect_mekan,aspect_servis,aspect_tatlı,aspect_yemek
0,her gittigimde bir öncekinden daha kötü susi ...,0,0,0,1,0,0,0,0,0,0
1,herkesin muhallebisini ya da süt tatlilari ye...,0,0,0,0,0,0,0,0,1,0
2,yogun oldugu saatlerde servis biraz aksak ols...,0,0,0,0,0,0,0,1,0,0
3,100 lira fix menuye gittik 8 kisi 1600 tl oded...,0,0,1,0,0,0,0,0,0,0
4,500 kişiye kaliteli servis verilemiyorsa 300 k...,0,0,0,0,0,0,0,1,0,0


In [ ]:
aspect_columns = ['aspect_döner', 'aspect_et', 'aspect_fiyat', 'aspect_içecek', 'aspect_kahvalti',
                  'aspect_manzara', 'aspect_mekan', 'aspect_servis', 'aspect_tatlı', 'aspect_yemek']

df_test[aspect_columns] = df_test[aspect_columns].replace(2, 1)



In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             83 non-null     object
 1   aspect_döner     83 non-null     uint8 
 2   aspect_et        83 non-null     uint8 
 3   aspect_fiyat     83 non-null     uint8 
 4   aspect_içecek    83 non-null     uint8 
 5   aspect_kahvalti  83 non-null     uint8 
 6   aspect_manzara   83 non-null     uint8 
 7   aspect_mekan     83 non-null     uint8 
 8   aspect_servis    83 non-null     uint8 
 9   aspect_tatlı     83 non-null     uint8 
 10  aspect_yemek     83 non-null     uint8 
dtypes: object(1), uint8(10)
memory usage: 1.6+ KB


9'a düşürmek için bir aspecti daha siliyorum test setinden.

In [ ]:
df_test.drop('aspect_kahvalti',axis=1,inplace=True)

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            83 non-null     object
 1   aspect_döner    83 non-null     uint8 
 2   aspect_et       83 non-null     uint8 
 3   aspect_fiyat    83 non-null     uint8 
 4   aspect_içecek   83 non-null     uint8 
 5   aspect_manzara  83 non-null     uint8 
 6   aspect_mekan    83 non-null     uint8 
 7   aspect_servis   83 non-null     uint8 
 8   aspect_tatlı    83 non-null     uint8 
 9   aspect_yemek    83 non-null     uint8 
dtypes: object(1), uint8(9)
memory usage: 1.5+ KB


In [ ]:
df_test.head()

,text,aspect_döner,aspect_et,aspect_fiyat,aspect_içecek,aspect_manzara,aspect_mekan,aspect_servis,aspect_tatlı,aspect_yemek
0,her gittigimde bir öncekinden daha kötü susi ...,0,0,0,1,0,0,0,0,0
1,herkesin muhallebisini ya da süt tatlilari ye...,0,0,0,0,0,0,0,1,0
2,yogun oldugu saatlerde servis biraz aksak ols...,0,0,0,0,0,0,1,0,0
3,100 lira fix menuye gittik 8 kisi 1600 tl oded...,0,0,1,0,0,0,0,0,0
4,500 kişiye kaliteli servis verilemiyorsa 300 k...,0,0,0,0,0,0,1,0,0


In [ ]:
sequences = tokenizer.texts_to_sequences(df_test['text'])

vocab_size2=tokenizer.num_words
embedding_dim=100
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
targets = df_test.iloc[:, 1:].to_numpy()

In [ ]:
print(targets.shape)


(83, 9)


In [ ]:
y_pred = model.predict(padded_sequences)

threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)
y_pred_binary_reshaped = np.reshape(y_pred_binary, (y_pred_binary.shape[0], -1))[:, :9]


3/3 [==============================] - 0s 6ms/step


In [ ]:
targets.shape

(83, 9)

In [ ]:
y_pred_binary_reshaped.shape

(83, 9)

In [ ]:


threshold = 0.5
y_pred_binary_reshaped = (y_pred_binary_reshaped > threshold).astype(int)

precision = precision_score(targets, y_pred_binary_reshaped, average='micro')
recall = recall_score(targets, y_pred_binary_reshaped, average='micro')

print("Precision:", precision)
print("Recall:", recall)


Precision: 0.19642857142857142
Recall: 0.12087912087912088


As expected, the test set also yielded a relatively low result. As I mentioned before, I believe this is mainly due to the small size of the dataset.